# Ch.3
## 3.1 Tensorboard
*Tensorboardを利用するために必要なプロセス*  
  
1. モデル構築時に必要なところでログを取得するオペレーションを記載する。
2. 全てログを配置し終わったら、そのログをマージするオペレーションを記載する。
3. `tf.summary.FireWriter`を呼び出してログの出力先と対象グラフを指定する。
4. ログを取得するタイミングでマージするオペレーションを実行する。
5. 実行結果を`FireWriter`に追加する。
6. 出力されたログを`tensorboard`コマンドで指定して呼び出す。
7. ブラウザで対象サーバーにアクセスする。

### Images
バイナリデータ化した画像がモデルに正しく渡っているか、オーギュメンテーション後の画像が判別可能なレベルかなどを確認するのに有効。  
input imageが[mini batch size, 784]なので、[mini batch size, hight, width, channel]に変換して表示させる。  
  
### Graph
計算グラフを可視化する`Graph`は、特にログを取得するオペレーションを記載する必要がない。  
`FireWriter`を宣言する際に対象のグラフを指定してログ出力ディレクトリを指定した時点で勝手にログが作成される（後述）。  
  
### name_scope  
TensorFlowではシンプルな構成のネットワークでもオペレーションの数が多くなることが少なくない。  
しかし、そのままGraphを表示すると、全てのオペレーションが表示されてしまい可読性が下がる。  
そこで、`name_scope`を指定して各層や特定の処理ごとにまとめると、整理されてわかりやすくなる。  
### Scalars 
`loss`と`accuracy`の処理の後に`Scalars`を配置する。  
`tf.summary.scalar()`の入力値はスカラー（0階テンソル）であることに注意。  
→重みやバイアスはスカラーではないので、`Scalars`が使えない。  
→そういう場合は`tf.summary.histgram()`を利用する。  
  
### Distributions / Histogram  
引数の制約はほとんど無く、数値データのテンソルであれば何でも構わない。  
ただし、NeuralNetworkが発散した場合に`Ｎａｎ`となる場合があり、`Ｎａｎ`がひとつでもあるとエラーを吐くので注意。  
`tf.summary.histogram()`に値があれば、TensorBoardでは`Histograms`と`Distributions`の両方で結果を見ることができる。  
- Distributions:  確率分布/推移を時間軸ごとに見ることができる  
- Histogram:　ステップの度数分布を確認できる  
  
### Merging  
ログのマージには2種類あり、一部のログをマージするものと、全てのログをマージするもの。  
当たり前だが、後者は全てのログの配置が終わった後にマージすること。  

```
# merging some logs
merge = tf.summary.merge([a, b, c])

# merging all the logs
summary_op = tf.summary.merge_all()
```

### FileWriter
取得したログを実際にログファイルに書き出すのが`tf.summary.FireWriter`で、宣言するだけで計算グラフを`Graphs`に出力してくれる。  
第一引数はログ出力対象のディレクトリで、絶対パスでも相対パスでも書ける。  
  
  
### TensorBoard  
TensorBoardを起動するには、コマンドラインから以下のコマンドを実行する。  
（`path/to/log-directory`には実際にログが保存されているパスを入力）  
  
`tensorboard --logdir /path/to/log-directory`  
  
起動後は以下のURLからアクセスできる。  
http://localhost:6006

## mnist_tb()について
1行あたりの長さを気にせず書いたので、やはり見にくい。  
下の`mnist_cnn()`と比べると読みづらさがわかる。

In [1]:
#!/usr/bin/env python
# coding=utf-8

import tensorflow as tf

def mnist_tb():
    # importing MNIST data
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets('mnist_data/', one_hot=True)

    # getting train images, labeled data as batch size = 50
    train_images, train_labels = mnist.train.next_batch(50)

    # getting all the test images
    test_images = mnist.test.images

    # getting all the test labels
    test_labels = mnist.test.labels

    # input layer
    # using None for an arbitrary batch size
    # this is really useful when train and validation batches
    # are different size
    x = tf.placeholder(tf.float32, [None, 784])

    # getting a log of the input image
    # x is [None, 784] and need a transformation into an image
    # img: [size of the mini batch, hight, width, channel]
    img = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input_data', img, 10)

    # hidden layer
    with tf.name_scope('hidden'):
        w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name='w1')
        b_1 = tf.Variable(tf.zeros([64]), name='b1')
        h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)

        # logging
        tf.summary.histogram('w_1', w_1)

    # output layer
    with tf.name_scope('output'):
        w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name='w2')
        b_2 = tf.Variable(tf.zeros([10]), name='b2')
        out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)

    # loss function
    # computing mean with tf.reduce_mean
    y = tf.placeholder(tf.float32, [None, 10])
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.square(y - out))
        # logging
        tf.summary.scalar('loss', loss)

    # train
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # evaluation
    with tf.name_scope('accuracy'):
        correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        # logging
        tf.summary.scalar('accuracy', accuracy)

    # merging all the logs
    summary_op = tf.summary.merge_all()

    # defining initializer
    init = tf.global_variables_initializer()

    with tf.Session() as sess:

        sess.run(init)

        # FileWriter
        summary_writer = tf.summary.FileWriter('logs_mnist_tb', sess.graph)

        # loading test data
        test_images = mnist.test.images
        test_labels = mnist.test.labels

        for i in range(1000):
            step = i + 1
            train_images, train_labels = mnist.train.next_batch(50)
            sess.run(train_step, feed_dict={x:train_images, y:train_labels})

            if step % 10 == 0:
                # logging (outputs are protocol buffers of logs)
                summary_str = sess.run(summary_op,
                                       feed_dict={x:test_images, y:test_labels})
                # writing protocol buffers of logs
                summary_writer.add_summary(summary_str, step)

                # computing and printing accuracy
                acc_val = sess.run(accuracy,
                                   feed_dict={x:test_images, y:test_labels})
                print('Step {}: accuracy = {}'.format(step, acc_val))

/home/tk/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist_tb()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Step 10: accuracy = 0.15309999883174896
Step 20: accuracy = 0.17790000140666962
Step 30: accuracy = 0.2354000061750412
Step 40: accuracy = 0.28139999508857727
Step 50: accuracy = 0.3224000036716461
Step 60: accuracy = 0.36329999566078186
Step 70: accuracy = 0.39910000562667847
Step 80: accuracy = 0.4

## 3.3 Changing for a better neural network
  
In this section we will improve the network we made above by the following actions:  
  
- changing to a multi-layer network   
- introducing a convolutional neural network  
- setting *cross entropy* as the loss function 
  
今までのモデル：多層パーセプトロン  
→左上からピクセルが白か黒かということを精査して数字を判別しているだけ  
→これでは「どこに丸みがある」「どこに角がある」などの画像の特徴を利用していない  
→そこで特徴を抽出してそれを基に分類するCNNを導入する  
  
CNNの説明は省略。  
復習したい場合は[このブログ](https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/)か[この動画](https://youtu.be/2-Ol7ZB0MmU)がオススメ。  
  
クロスエントロピーには微小量を加算しないと、log0に限りなく近い値を取り、ニューラルネットワークが発散してしまう場合があるので要注意。  
  

In [7]:
#!/usr/bin/env python
# coding=utf-8

import tensorflow as tf

def mnist_cnn():
    # importing MNIST data
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets('mnist_data/', one_hot=True)

    # getting train images, labeled data as batch size = 50
    train_images, train_labels = mnist.train.next_batch(50)

    # getting all the test images
    test_images = mnist.test.images

    # getting all the test labels
    test_labels = mnist.test.labels

    # input layer
    # using None for an arbitrary batch size
    # this is really useful when train and validation batches
    # are different size
    x = tf.placeholder(tf.float32, [None, 784])

    # getting a log of the input image
    # x is [None, 784] and need a transformation into an image
    # img: [size of the mini batch, hight, width, channel]
    img = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input_data', img, 10)

    # conv1 layer
    #
    # Architecture:
    #  input: [1, 28, 28, 1]
    #  output: [1, 14, 14, 32]
    #  where [batch, hight, width, channels]
    #
    # Prams:
    #  f1: [hight, width, channels, num of filters (channels after convolution)]
    #  strides: [batch, hight, width, channels]
    #
    # Filters:
    #  filters in CNN correspond to weights in neural networks,
    #  so use tf.Variable as defining filters
    #  f: [hight, width, channels, num of filters (channels after convolution)]
    #  usually initialized with tf.truncated_normal()
    with tf.name_scope('conv_block1'):
        f1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32],
                                             stddev=0.1), name='f1')
        conv1 = tf.nn.conv2d(img, f1, strides=[1, 1, 1, 1],
                             padding='SAME', name='conv1')
        b1 = tf.Variable(tf.constant(0.1, shape=[32], name='b1'))
        h_conv1 = tf.nn.relu(conv1 + b1)
        h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1], padding='SAME')

        # logging
        tf.summary.histogram('f1', f1)
        tf.summary.histogram('b1', b1)

    # conv2 layer
    #
    # Architecture:
    #  input: [1, 14, 14, 32]
    #  output: [1, 7, 7, 64]
    #  where [batch, hight, width, channels]
    with tf.name_scope('conv_block2'):
        f2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64],
                                             stddev=0.1), name='f2')
        conv2 = tf.nn.conv2d(h_pool1, f2, strides=[1, 1, 1, 1],
                             padding='SAME', name='conv2')
        b2 = tf.Variable(tf.constant(0.1, shape=[64], name='b2'))
        h_conv2 = tf.nn.relu(conv2 + b2)
        h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1], padding='SAME')

        # logging
        tf.summary.histogram('f2', f2)
        tf.summary.histogram('b2', b2)

   # fully connected layer
    with tf.name_scope('fully_connected_layer'):
        # flattening the convoluted output
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

        # fully connected layer
        w_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024],
                                                stddev=0.1), name='w_fc1')
        b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]), name='b_fc1')
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

        # output layer
        w_fc2 = tf.Variable(tf.truncated_normal([1024, 10],
                                                stddev=0.1), name='w_fc2')
        b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]), name='b_fc2')
        out = tf.nn.softmax(tf.matmul(h_fc1, w_fc2) + b_fc2)

    # loss function
    # using cross entropy as loss function
    y = tf.placeholder(tf.float32, [None, 10])
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(out + 1e-5), axis=[1]))

        # logging
        tf.summary.scalar('loss', loss)

    # train
    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    # evaluation
    with tf.name_scope('accuracy'):
        correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        # logging
        tf.summary.scalar('accuracy', accuracy)

    # merging all the logs
    summary_op = tf.summary.merge_all()

    # defining initializer
    init = tf.global_variables_initializer()

    # excecution
    with tf.Session() as sess:
        sess.run(init)

        # FileWriter
        summary_writer = tf.summary.FileWriter('logs_mnist_cnn', sess.graph)

        # loading test data
        test_images = mnist.test.images
        test_labels = mnist.test.labels

        for i in range(1000):
            step = i + 1
            train_images, train_labels = mnist.train.next_batch(50)
            sess.run(train_step, feed_dict={x:train_images, y:train_labels})

            if step % 10 == 0:
                # logging (outputs are protocol buffers of logs)
                summary_str = sess.run(summary_op,
                                       feed_dict={x:test_images, y:test_labels})
                # writing protocol buffers of logs
                summary_writer.add_summary(summary_str, step)

                # computing and printing accuracy
                acc_val = sess.run(accuracy,
                                   feed_dict={x:test_images, y:test_labels})
                print('Step {}: accuracy = {}'.format(step, acc_val))

`mnist_cnn()` killed my kernel, so I recommend you to reset the kernel after excecuting every model or just excecute on AWS.

In [ ]:
mnist_cnn()

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [1]:
#!/usr/bin/env python
# coding=utf-8

import tensorflow as tf

# reduce_sum() examples
x = tf.constant([[1, 1, 1], [1, 1, 1]])

with tf.Session() as sess:
    print('x: ', sess.run(x))
    print('tf.reduce_sum(x): ',
          sess.run(tf.reduce_sum(x)))  # 6
    print('tf.reduce_sum(x, 0): ',
          sess.run(tf.reduce_sum(x, 0)))  # [2, 2, 2]
    print('tf.reduce_sum(x, 1): ',
          sess.run(tf.reduce_sum(x, 1)))  # [3, 3]
    print('tf.reduce_sum(x, 1, keepdims=True): ',
          sess.run(tf.reduce_sum(x, 1, keepdims=True)))  # [[3], [3]]
    print('tf.reduce_sum(x, [0, 1]): ',
          sess.run(tf.reduce_sum(x, [0, 1])))  # 6

/home/tk/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


x:  [[1 1 1]
 [1 1 1]]
tf.reduce_sum(x):  6
tf.reduce_sum(x, 0):  [2 2 2]
tf.reduce_sum(x, 1):  [3 3]
tf.reduce_sum(x, 1, keepdims=True):  [[3]
 [3]]
tf.reduce_sum(x, [0, 1]):  6


## 3.4 Saver
### Saving
モデルの保存にはいくつか方法があるが、以下の方法が一番シンプル。  
まずはセッションの前に`saver`を定義する。  

In [ ]:
######### THIS DOSEN'T WORK ###########

# saving all the models and variables
saver = tf.train.Saver()

# list is available
saver = tf.train.Saver([w1, w1])

# dict is also available
saver = tf.train.Saver({'param1':w1, 'param2':w2})

次に学習が終わった後に`saver.save(sess, 'PATH/NAME')`で`saver`を実行する。  
この方法だと全ての学習が終わった後のモデルしか保存できないが、実際には一定回数ごとにモデルを保存しておき、一番精度が高かったモデルを利用することが多い。  
そのためには、`acc_val`のスコープ内で`saver`を実行し、ループさせる必要がある。  
このとき、`global_step`引数でステップ数を指定する。  
  
モデルの保存には容量が必要なので、保存数に上限が設けられていることに注意する。  
`max_to_keep`でこれを指定しており、デフォルトは5。  
  
また、グラフの構造などを記述するmetaファイルも毎回書き込みが行われるが、基本的には学習中に用いるモデルが不変なので、この書き込みは省いたほうが良い。  
しかもmetaファイルはモデルの設計コードがなくてもモデルを読み込むための復元ファイルみたいなものなので、今回のように手元にソースコードがある場合は不要である。  
そういう場合は`write_meta_graph=False`と指定する。  
  
如何に計算を効率化しても、このようなlog出力やチェックポイントファイルの出力がボトルネックになって学習が進まないということが多々あるので、実際に運用する際には気を付ける。

### global_step
上記に加えて、学習を中断した際に、正しいステップ数から再開するために`global_step`を正しく指定する必要がある。  
この設定をせずに再開できないモデルを設計してしまうと、TensorBoardと組み合わせて利用したい時に困るらしい。  
そこで、`global_step`もモデル内に保存し、環境が変わってもステップ数を読み込めるようにする。

In [ ]:
# train
global_step = tf.Variable(0, name='global_step', trainable=False)
train_step = tf.train.Gradient.DescentOptimizer(0.5)\
    .minimize(loss, global_step=global_step)

`minimize`メソッドの第二引数に`global_step`を指定することで、trainごとにincrementしてくれるようになる。
`trainable=False`は、計算グラフの訓練対象の`Variable()`コレクションに`global_step`を加えないための措置。  
モデルが複雑になると、訓練用の変数を全て取得して処理を行うこともあるので、習慣的に`global_step`は`trainable=False`にする。  
あとはtrain loop前に`global_step`を実行し、ステップ数を計算してからtrainを実行させる。  
最終的なコードは以下の通り。

In [1]:
#!/usr/bin/env python
# coding=utf-8

import tensorflow as tf

def mnist_saver():
    # importing MNIST data
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets('mnist_data/', one_hot=True)

    # getting train images, labeled data as batch size = 50
    train_images, train_labels = mnist.train.next_batch(50)

    # getting all the test images
    test_images = mnist.test.images

    # getting all the test labels
    test_labels = mnist.test.labels

    # input layer
    # using None for an arbitrary batch size
    # this is really useful when train and validation batches
    # are different size
    x = tf.placeholder(tf.float32, [None, 784])

    # getting a log of the input image
    # x is [None, 784] and need a transformation into an image
    # img: [size of the mini batch, hight, width, channel]
    img = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input_data', img, 10)

    # conv1 layer
    #
    # Architecture:
    #  input: [1, 28, 28, 1]
    #  output: [1, 14, 14, 32]
    #  where [batch, hight, width, channels]
    #
    # Prams:
    #  f1: [hight, width, channels, num of filters (channels after convolution)]
    #  strides: [batch, hight, width, channels]
    #
    # Filters:
    #  filters in CNN correspond to weights in neural networks,
    #  so use tf.Variable as defining filters
    #  f: [hight, width, channels, num of filters (channels after convolution)]
    #  usually initialized with tf.truncated_normal()
    with tf.name_scope('conv_block1'):
        f1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32],
                                             stddev=0.1), name='f1')
        conv1 = tf.nn.conv2d(img, f1, strides=[1, 1, 1, 1],
                             padding='SAME', name='conv1')
        b1 = tf.Variable(tf.constant(0.1, shape=[32], name='b1'))
        h_conv1 = tf.nn.relu(conv1 + b1)
        h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1], padding='SAME')

        # logging
        tf.summary.histogram('f1', f1)
        tf.summary.histogram('b1', b1)

    # conv2 layer
    #
    # Architecture:
    #  input: [1, 14, 14, 32]
    #  output: [1, 7, 7, 64]
    #  where [batch, hight, width, channels]
    with tf.name_scope('conv_block2'):
        f2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64],
                                             stddev=0.1), name='f2')
        conv2 = tf.nn.conv2d(h_pool1, f2, strides=[1, 1, 1, 1],
                             padding='SAME', name='conv2')
        b2 = tf.Variable(tf.constant(0.1, shape=[64], name='b2'))
        h_conv2 = tf.nn.relu(conv2 + b2)
        h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                                 strides=[1, 2, 2, 1], padding='SAME')

        # logging
        tf.summary.histogram('f2', f2)
        tf.summary.histogram('b2', b2)

   # fully connected layer
    with tf.name_scope('fully_connected_layer'):
        # flattening the convoluted output
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

        # fully connected layer
        w_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024],
                                                stddev=0.1), name='w_fc1')
        b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]), name='b_fc1')
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

        # output layer
        w_fc2 = tf.Variable(tf.truncated_normal([1024, 10],
                                                stddev=0.1), name='w_fc2')
        b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]), name='b_fc2')
        out = tf.nn.softmax(tf.matmul(h_fc1, w_fc2) + b_fc2)

    # loss function
    # using cross entropy as loss function
    y = tf.placeholder(tf.float32, [None, 10])
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(out + 1e-5), axis=[1]))

        # logging
        tf.summary.scalar('loss', loss)

    # train
    with tf.name_scope('train'):
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_step = tf.train.GradientDescentOptimizer(0.01)\
            .minimize(loss, global_step=global_step)

    # evaluation
    with tf.name_scope('accuracy'):
        correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        # logging
        tf.summary.scalar('accuracy', accuracy)

    # merging all the logs
    summary_op = tf.summary.merge_all()

    # defining initializer
    init = tf.global_variables_initializer()

    # defining saver
    saver = tf.train.Saver(max_to_keep=3)

    # excecution
    with tf.Session() as sess:
        sess.run(init)

        # FileWriter
        summary_writer = tf.summary.FileWriter('logs_mnist_saver', sess.graph)

        # loading test data
        test_images = mnist.test.images
        test_labels = mnist.test.labels

        # loading global_step
        last_step = sess.run(global_step)

        for i in range(1000):
            step = last_step + i
            train_images, train_labels = mnist.train.next_batch(50)
            sess.run(train_step, feed_dict={x:train_images, y:train_labels})

            if (step + 1) % 100 == 0:
                # logging (outputs are protocol buffers of logs)
                summary_str = sess.run(summary_op,
                                       feed_dict={x:test_images, y:test_labels})
                # writing protocol buffers of logs
                summary_writer.add_summary(summary_str, step)

                # computing and printing accuracy
                acc_val = sess.run(accuracy,
                                   feed_dict={x:test_images, y:test_labels})

                # showing the accuracy during training 
                print('Step {}: accuracy = {}'.format(step + 1, acc_val))

                # saving models
                saver.save(sess, 'ckpt/mnist_saver',
                           global_step=step + 1, write_meta_graph=False)

/home/tk/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist_saver()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Step 100: accuracy = 0.8689000010490417
Step 200: accuracy = 0.9211999773979187
Step 300: accuracy = 0.9406999945640564
Step 400: accuracy = 0.9422000050544739
Step 500: accuracy = 0.9441999793052673
Step 600: accuracy = 0.9484000205993652
Step 700: accuracy = 0.9613999724388123
Step 800: accuracy = 

### Restoring the model

In [ ]:
with tf.Session() as sess:
    ckpt_state = tf.train.get_checkpoint_state('ckpt/')
    if ckpt_state:
        last_model = ckpt_state.model_checkpoint_path
        saver.restore(sess, last_model)
        print('model was loaded:', last_model)
    else:
        sess.run(init)
        print('initialized.')

`last_model = ckpt_state.model_checkpoint_path`だと最新のモデルを読み込むことになるので、もしも任意のモデルを読み込みたい場合は以下のように指定する。

In [ ]:
model = ckpt_state.all_model_checkpoint_paths[0] # the latest
model = ckpt_state.all_model_checkpoint_paths[1] # the 2nd latest
model = ckpt_state.all_model_checkpoint_paths[2] # the 3rd latest